# Assignment 4
Anushka Deshpande \
USC ID: 5914892345

## Questions:

Task 1: 
1. What are the precision, recall and F1 scores on dev data?\
precision:  78.63%; recall:  72.50%; F-1 score:  75.44


Task 2:
1. How to deal with the conflict of capitalisaation? \
After trying multiple ways of dealing with capital words, including boolean mask, it was observed that ignoring the case of the word returns the best trained model. Hence, for this submission , the case of the word has been ignored.

2. What are the precision, recall and F1 scores on dev data?\
precision:  88.27%; recall:  89.38%; FB1:  88.82

## Importing Libraries

Here we are importing the required libraries into the program. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/HW4')

In [ ]:
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
import random
import json
torch.manual_seed(0)
random.seed(0)

Here, we are setting the device to cuda if available to make use of maximum computation power available to us.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Below is a class, createData. It contains the following functions detailed below:

1. SplitTrainXY: \
This function splits the train data into x_train and y_train. This function is also called to split validation data into x_val and y_val.

2. readFileTrain: \
This function reads the train data from the file, calls the SplitTrainXY function to split the data and returns x_train and y_train. It is also used to obtain x_val and y_val from validation data.

3. splitTestX: \
This function gives the test data in the required format. It matches the data format of x_test with x_train. 

4. readFileTest: \
This funciton reads the test data from the data file and calls splitTestX on the test data, and returns x_test.


In [ ]:
class createData():
    
  def SplitTrainXY(self, dataset):
      x_train, y_train = list(), list()
      x, y = list(), list()
      first = 1
      for entry in dataset.itertuples():
          # print(type(entry.id))
          # break
          if(entry.ID == '1' and first == 0):
              x_train.append(x)
              y_train.append(y)
              x = list()
              y = list()
          first = 0
          x.append(entry.Word)
          y.append(entry.NERTag)

      x_train.append(x)
      y_train.append(y)

      return x_train, y_train

  def readFileTrain(self, filepath):
      train_df = list()
      with open(filepath, 'r') as inputFile:
          for line in inputFile.readlines():
              if len(line) > 2:
                id, word, ner_tag = line.strip().split(" ")
                train_df.append([id, word, ner_tag])

      train_df = pd.DataFrame(train_df, columns=['ID', 'Word', 'NERTag'])
      train_df = train_df.dropna()
      x_train, y_train = self.SplitTrainXY(train_df)
      return x_train, y_train

  def SplitTestX(self, dataset):
      x_train = list()
      x = list()
      first = 1
      for entry in dataset.itertuples():
          # print(type(entry.id))
          # break
          if(entry.ID == '1' and first == 0):
              x_train.append(x)
              x = list()
          first = 0
          x.append(entry.Word)

      x_train.append(x)
      return x_train

  def readFileTest(self, filepath):
      train_df = list()
      with open(filepath, 'r') as inputFile:
          for line in inputFile.readlines():
              if len(line) > 1:
                  id, word = line.strip().split(" ")
                  train_df.append([id, word])

      train_df = pd.DataFrame(train_df, columns=['ID', 'Word'])
      train_df = train_df.dropna()
      x_train = self.SplitTestX(train_df)
      return x_train

obj = createData()
x_train, y_train = obj.readFileTrain('./data/train')
x_val, y_val = obj.readFileTrain('./data/dev')
x_test = obj.readFileTest('./data/test')

print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test))

14987 14987
3466 3466
3684


Here we have a class getVectors() which is used to generate and return the various vectors and matrices required for training the BiLSTM models. The variou functions are described as follows:

1. prepareVocabulary(): \
This function prepares a list of unique words in the entire dataset, and returns it as a set. 

2. prepareLabelDictionary(): \
This function prepares label tags for each word from each sentence in the training data. The output of this function is a matrix of labels for each word in each sentence in the dataset.

3. prepareWordIndex(): \
This function prepares a dictionary of all the words available and gives them an index. This is stored in the form of a dictionary of the form a word-index pair.

4. vectorizeSentence(): \
This function is used to convert the train data from word wise, to sentence wise format. Each entry in the returned data is the vectors for each word in one sentence on the data.

5. vectorizeLabel(): \
This function prepares label tags for each word in each sentence.

6. createEmbedMatrix(): \
This function creates and returns the embedding matrix for Glove data. 

7. initialiseClassWeights(): \
This function initialises and returns the class weights to be assigned to the model during initialisation. 

In [ ]:
class getVectors():

  def prepareVocabulary(self, dataset):
      vocab = set()
      for sentence in dataset:
          for word in sentence:
              vocab.add(word)
      return vocab
  
  def prepareLabelDictionary(self, train_y, val_y):
      label1 = self.prepareVocabulary(train_y)
      label2 = self.prepareVocabulary(val_y)
      label = label1.union(label2)
      label_tuples = []
      counter = 0
      for tags in label:
          label_tuples.append((tags, counter))
          counter += 1
      label_dict = dict(label_tuples)
      return label_dict

  def prepareWordIndex(self, train, val):
      wordIdx = {"<PAD>": 0, "<UNK>": 1}
      idx = 2
      for data in [train, val]:
          for sent in data:
              for word in sent:
                  if word not in wordIdx:
                      wordIdx[word] = idx
                      idx += 1
      return wordIdx

  def vectorizeSentence(self, train, wordIdx):
      train_vec = list()
      tmp_x = list()
      for words in train:
          for word in words:
            if word not in wordIdx.keys():
              tmp_x.append(wordIdx['<UNK>'])
            else:
              tmp_x.append(wordIdx[word])
          train_vec.append(tmp_x)
          tmp_x = list()
      return train_vec

  def vectorizeLabel(self, train_y, label_dict):
      train_y_vec = list()
      for tags in train_y:
          tmp_yy = list()
          for label in tags:
              tmp_yy.append(label_dict[label])
          train_y_vec.append(tmp_yy)
      return train_y_vec
  
  def createEmbedMatrix(self, wordIdx, embedDictionary, dimension):
      embedMatrix = np.zeros((len(wordIdx), dimension))
      for word, idx in wordIdx.items():
          if word in embedDictionary:
              embedMatrix[idx] = embedDictionary[word]
          else:
              if word.lower() in embedDictionary:
                  embedMatrix[idx] = embedDictionary[word.lower()] + 5e-3
              else:
                  embedMatrix[idx] = embedDictionary["<UNK>"]

      return embedMatrix
      
  def initializeClassWeights(self, label_dict, train_y, val_y):
    classWeights = dict()
    for key in label_dict:
        classWeights[key] = 0
    total_nm_tags = 0
    for data in [train_y, val_y]:
        for tags in data:
            for tag in tags:
                total_nm_tags += 1
                classWeights[tag] += 1

    classWt = list()
    for key in classWeights.keys():
        if classWeights[key]:
            score = round(math.log(0.35*total_nm_tags / classWeights[key]), 2)
            classWeights[key] = score if score > 1.0 else 1.0
        else:
            classWeights[key] = 1.0
        classWt.append(classWeights[key])
    classWt = torch.tensor(classWt)
    return classWt


obj2 = getVectors()
word_idx = obj2.prepareWordIndex(x_train, x_val)
x_train_vectors = obj2.vectorizeSentence(x_train, word_idx)
x_test_vectors = obj2.vectorizeSentence(x_test, word_idx)
x_val_vectors = obj2.vectorizeSentence(x_val, word_idx)
label_dictionary = obj2.prepareLabelDictionary(y_train, y_val)
y_train_vectors = obj2.vectorizeLabel(y_train, label_dictionary)
y_val_vectors = obj2.vectorizeLabel(y_val, label_dictionary)

class_wt = obj2.initializeClassWeights(label_dictionary, y_train, y_val)

## Task 1: Simple Bidirectional LSTM

The class below creates a BiLSTM model with the following parameters and architecture:

Architecture: Embedding layer --> LSTM --> Linear layer --> ELU --> Classifier

Parameters: \
1. Embedding Dimension: 100
2. Number of LSTM layers: 1
3. LSTM Hidden Dimensions: 256
4. LSTM Dropout: 0.33
5. Linear Layer Output Dimensions: 128

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, vocabSize, embedDimensions, linearOutputDimension, hiddenLayerDimension, LSTMLayers,
                 bidirectional, dropoutValue, tagSize):
        super(BiLSTM, self).__init__()
        """ Hyper Parameters """
        self.hiddenLayerDimension = hiddenLayerDimension 
        self.LSTMLayers = LSTMLayers 
        self.embedDimensions = embedDimensions 
        self.linearOutputDimension = linearOutputDimension  
        self.tagSize = tagSize 
        if bidirectional:
          self.num_directions = 2 
        else:
          self.num_directions = 1

        """ Initializing Network """
        #Embedding Layer
        self.embedding = nn.Embedding(
            vocabSize, embedDimensions) 
        self.embedding.weight.data.uniform_(-1,1)

        #BiLSTM Layer
        self.LSTM = nn.LSTM(embedDimensions,
                            hiddenLayerDimension,
                            num_layers=LSTMLayers,
                            batch_first=True,
                            bidirectional=True)
        
        #Linear Layer
        self.fc = nn.Linear(hiddenLayerDimension*self.num_directions,
                            linearOutputDimension)  
        
        #Dropout Layer
        self.dropout = nn.Dropout(dropoutValue)

        #ELU Layer
        self.elu = nn.ELU(alpha=0.01)

        #Classifier 
        self.classifier = nn.Linear(linearOutputDimension, self.tagSize)

    def init_hidden(self, batchSize):
        h, c = (torch.zeros(self.LSTMLayers * self.num_directions,
                            batchSize, self.hiddenLayerDimension).to(device),
                torch.zeros(self.LSTMLayers * self.num_directions,
                            batchSize, self.hiddenLayerDimension).to(device))
        return h, c

    def forward(self, sentence, sentenceLength):  
        # Set initial states
        batchSize = sentence.shape[0]
        h_0, c_0 = self.init_hidden(batchSize)

        # Forward propagate LSTM
        embedded = self.embedding(sentence).float()
        packed_embedded = pack_padded_sequence(
            embedded, sentenceLength, batch_first=True, enforce_sorted=False)
        output, _ = self.LSTM(packed_embedded, (h_0, c_0))
        output_unpacked, _ = pad_packed_sequence(output, batch_first=True)
        dropout = self.dropout(output_unpacked)
        linear = self.fc(dropout)
        prediction = self.elu(linear)
        prediction = self.classifier(prediction)
        return prediction


## Task 2: Glove word Embeddings

The class below creates a BiLSTM model with the glove vectors with the following parameters and architecture: 

Architecture: Embedding layer --> LSTM --> Linear layer --> ELU --> Classifier

Parameters: \
1. Embedding Dimension: 100
2. Number of LSTM layers: 1
3. LSTM Hidden Dimensions: 256
4. LSTM Dropout: 0.33
5. Linear Layer Output Dimensions: 128

In [ ]:
class BiLSTM_Glove(nn.Module):
    def __init__(self, vocabSize, embedDimension, linearOutputDimension, hiddenLayerDimension, LSTMLayers,
                 bidirectional, dropoutValue, tagSize, embedMatrix):
        super(BiLSTM_Glove, self).__init__()
        """ Hyper Parameters """
        self.hiddenLayerDimension = hiddenLayerDimension 
        self.LSTMLayers = LSTMLayers 
        self.embedDimension = embedDimension 
        self.linearOutputDimension = linearOutputDimension 
        self.tagSize = tagSize 
        self.embedMatrix = embedMatrix
        if bidirectional:
          self.numDirections = 2 
        else:
          self.numDirections = 1

        """ Initializing Network """
        #Embedding layer
        self.embedding = nn.Embedding(vocabSize, embedDimension)  
        self.embedding.weight = nn.Parameter(torch.tensor(embedMatrix))

        #LSTM Layer
        self.LSTM = nn.LSTM(embedDimension,
                            hiddenLayerDimension,
                            num_layers=LSTMLayers,
                            batch_first=True,
                            bidirectional=True)
        
        #Linear layer
        self.fc = nn.Linear(hiddenLayerDimension*self.numDirections, linearOutputDimension) 

        #Dropout layer
        self.dropout = nn.Dropout(dropoutValue)

        #ELU Layer
        self.elu = nn.ELU(alpha=0.01)

        #CLassifier
        self.classifier = nn.Linear(linearOutputDimension, self.tagSize)

    def init_hidden(self, batch_size):
        h, c = (torch.zeros(self.LSTMLayers * self.numDirections,
                            batch_size, self.hiddenLayerDimension).to(device),
                torch.zeros(self.LSTMLayers * self.numDirections,
                            batch_size, self.hiddenLayerDimension).to(device))
        return h, c

    def forward(self, sentence, sentenceLength):  
        #Set initial states
        batch_size = sentence.shape[0]
        h_0, c_0 = self.init_hidden(batch_size)

        # Forward propagate LSTM
        embedded = self.embedding(sentence).float()
        packed_embedded = pack_padded_sequence(embedded, sentenceLength, batch_first=True, enforce_sorted=False)
        output, _ = self.LSTM(packed_embedded, (h_0, c_0))
        output_unpacked, _ = pad_packed_sequence(output, batch_first=True)
        dropout = self.dropout(output_unpacked)
        linear = self.fc(dropout)
        prediction = self.elu(linear)
        prediction = self.classifier(prediction)
        return prediction
    

The two classes below, BiLSTM_DataLoader and BiLSTM_TestLoader load the dataset by converting the data into torch tensors. 

In [ ]:
class BiLSTM_DataLoader(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        xInstance = torch.tensor(self.x[index])  # , dtype=torch.long
        yInstance = torch.tensor(self.y[index])  # , dtype=torch.float
        return xInstance, yInstance

In [ ]:
class BiLSTM_TestLoader(Dataset):
    def __init__(self, x):
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        xInstance = torch.tensor(self.x[index])  # , dtype=torch.long
        return xInstance

The two classes below, CustomCollator and CustomTestCollator both implement padding for the input data by considering the max padding length as the max length in the Batch. 

In [ ]:
class CustomCollator(object):

    def __init__(self, vocabulary, label):
        self.params = vocabulary
        self.label = label

    def __call__(self, batch):
        (xx, yy) = zip(*batch)
        xLength = [len(x) for x in xx]
        ylength = [len(y) for y in yy]
        maxBatchlength = max([len(s) for s in xx])
        batchData = self.params['<PAD>']*np.ones((len(xx), maxBatchlength))
        batchLabels = -1*np.zeros((len(xx), maxBatchlength))
        for j in range(len(xx)):
            cur_len = len(xx[j])
            batchData[j][:cur_len] = xx[j]
            batchLabels[j][:cur_len] = yy[j]

        batchData, batchLabels = torch.LongTensor(
            batchData), torch.LongTensor(batchLabels)
        batchData, batchLabels = Variable(batchData), Variable(batchLabels)

        return batchData, batchLabels, xLength, ylength

In [ ]:
class CustomTestCollator(object):

    def __init__(self, vocabulary, label):
        self.params = vocabulary
        self.label = label

    def __call__(self, batch):
        xx = batch
        xLen = [len(x) for x in xx]
        maxBatchlength = max([len(s) for s in xx])
        batchData = self.params['<PAD>']*np.ones((len(xx), maxBatchlength))
        for j in range(len(xx)):
            cur_len = len(xx[j])
            batchData[j][:cur_len] = xx[j]

        batchData = torch.LongTensor(batchData)
        batchData = Variable(batchData)

        return batchData, xLen

## Training:

The class below is used for training the data. It has 2 functions, train_lstm, which trains the Simple BiLSTM model, and train_glove which trains the Glove model using the training and dev datasets. 

1. Simple BiLSTM Training: \
It is trained using the following parameters: \
  *   batch size: 4
  *   optimizer: SGD, with learning_rate = 0.1 and momentum = 0.9
  *   scheduler: StepLR, with step_size = 20 and gamma = 0.9

  This model is trained for 50 epochs and we achieve  
    * accuracy : 95.00
    * validation F1 score : 74.92



2. BiLSTM Training using Glove Embeddings: \
It is trained using the following parameters: \
  *   batch size: 8
  *   optimizer: SGD, with learning_rate = 0.1 and momentum = 0.9
  *   scheduler: StepLR, with step_size = 15 and gamma = 0.9

  This model is trained for 25 epochs and we achieve  
    * accuracy : 97.95
    * validation F1 score : 88.82

The dev outputs are stored in 2 files, dev1.out and dev2.out.





In [ ]:
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, MultiStepLR, ReduceLROnPlateau

class train():

  def train_lstm(self):
    BiLSTM_model = BiLSTM(vocabSize=len(word_idx),
                          embedDimensions=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary))
    BiLSTM_model.to(device)
    print(BiLSTM_model)

    BiLSTM_train = BiLSTM_DataLoader(x_train_vectors, y_train_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader = DataLoader(dataset=BiLSTM_train,
                            batch_size=4,
                            drop_last=True,
                            collate_fn=custom_collator)

    BiLSTM_dev = BiLSTM_DataLoader(x_val_vectors, y_val_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader_dev = DataLoader(dataset=BiLSTM_dev,
                                batch_size=1,
                                shuffle=False,
                                drop_last=True,
                                collate_fn=custom_collator)

    criteria = nn.CrossEntropyLoss(weight=class_wt)
    criteria = criteria.to(device)
    criteria.requres_grad = True
    optimizer = torch.optim.SGD(BiLSTM_model.parameters(), lr=0.1, momentum=0.9)
    scheduler = StepLR(optimizer, step_size = 20, gamma=0.9)
    epochs = 50

    for epoch in range(1, epochs+1):
        train_loss = 0.0
        for input, label, inputLength, labelLength in dataloader:
            optimizer.zero_grad()
            output = BiLSTM_model(input.to(device), inputLength)
            output = output.view(-1, len(label_dictionary))
            label = label.view(-1)
            loss = criteria(output, label.to(device))
            loss.backward()
            train_loss += loss.item() * input.size(1)
            optimizer.step()

        train_loss = train_loss / len(dataloader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        #torch.save(BiLSTM_model.state_dict(),'BiLSTM_b1_epoch_' + str(i) + '.pt')
        #BiLSTM_model.load_state_dict(torch.load("./BiLSTM_b1_epoch_"+str(e)+".pt"))#125
        BiLSTM_model.to(device)
        print(label_dictionary)
        revLabelDict = {v: k for k, v in label_dictionary.items()}
        revVocabDict = {v: k for k, v in word_idx.items()}

        devTemp = open("./devtemp.out", 'w')
        dev1 = open("./dev1.out", 'w')
        val_loss = 0.0
        for devData, label, devDataLength, labelDataLength in dataloader_dev:

            prediction = BiLSTM_model(devData.to(device), devDataLength)

            output = prediction.view(-1, len(label_dictionary))
            label2 = label.view(-1)
            loss = criteria(output, label2.to(device))
            loss.backward()
            val_loss += loss.item() * devData.size(1)

            prediction = prediction.cpu()
            prediction = prediction.detach().numpy()
            label = label.detach().numpy()
            devData = devData.detach().numpy()
            prediction = np.argmax(prediction, axis=2)
            prediction = prediction.reshape((len(label), -1))

            for i in range(len(devData)):
                for j in range(len(devData[i])):
                    if devData[i][j] != 0:
                        word = revVocabDict[devData[i][j]]
                        gold = revLabelDict[label[i][j]]
                        op = revLabelDict[prediction[i][j]]
                        devTemp.write(" ".join([str(j+1), word, gold, op]))
                        devTemp.write("\n")
                        dev1.write(" ".join([str(j+1), word, op]))
                        dev1.write("\n")
                devTemp.write("\n")        
                dev1.write("\n")
        devTemp.close()
        dev1.close()

        scheduler.step()

        if epoch == epochs:
          print('model saved')
          torch.save(BiLSTM_model.state_dict(),'blstm1.pt')

        !perl conll03eval.txt < devtemp.out

  def train_glove(self):

    glove = pd.read_csv('./glove.6B.100d.gz', sep=" ",
                        quoting=3, header=None, index_col=0)
    glove_emb = {key: val.values for key, val in glove.T.items()}

    word_idx = getVectors.prepareWordIndex(getVectors(), x_train, x_val)
    glove_vec = np.array([glove_emb[key] for key in glove_emb])
    glove_emb["<PAD>"] = np.zeros((100,), dtype="float64")
    glove_emb["<UNK>"] = np.mean(glove_vec, axis=0, keepdims=True).reshape(100,)
    emb_matrix = getVectors.createEmbedMatrix(getVectors(),
        wordIdx=word_idx, embedDictionary=glove_emb, dimension=100)

    vocab_size = emb_matrix.shape[0]
    vector_size = emb_matrix.shape[1]
    print(vocab_size, vector_size)
    Glove_Model = BiLSTM_Glove(vocabSize=len(word_idx),
                          embedDimension=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary),
                          embedMatrix=emb_matrix)
    Glove_Model.to(device)
    print(Glove_Model)

    BiLSTM_train = BiLSTM_DataLoader(x_train_vectors, y_train_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader = DataLoader(dataset=BiLSTM_train,
                            batch_size=8,
                            drop_last=True,
                            collate_fn=custom_collator)

    BiLSTM_dev = BiLSTM_DataLoader(x_val_vectors, y_val_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader_dev = DataLoader(dataset=BiLSTM_dev,
                                batch_size=1,
                                shuffle=False,
                                drop_last=True,
                                collate_fn=custom_collator)

    criterion = nn.CrossEntropyLoss(weight=class_wt)
    criterion = criterion.to(device)
    criterion.requres_grad = True
    optimizer = torch.optim.SGD(Glove_Model.parameters(), lr=0.1, momentum=0.9)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.9)
    epochs = 25

    for epoch in range(1, epochs+1):
        train_loss = 0.0
        for input, label, input_len, label_len in dataloader:
            optimizer.zero_grad()
            output = Glove_Model(input.to(device), input_len)
            output = output.view(-1, len(label_dictionary))
            label = label.view(-1)
            loss = criterion(output, label.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * input.size(1)

        train_loss = train_loss / len(dataloader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

        #torch.save(Glove_Model.state_dict(),'BiLSTM_glove_' + str(i) + '.pt')
        #Glove_Model.load_state_dict(torch.load("./BiLSTM_glove_"+str(e)+".pt"))#125

        Glove_Model.to(device)
        revLabelDict = {v: k for k, v in label_dictionary.items()}
        revVocabDict = {v: k for k, v in word_idx.items()}
        
        devTemp2 = open("devtemp2.out", 'w')
        dev2 = open("dev2.out", 'w')

        for devData, label, devDataLength, label_data_len in dataloader_dev:

            prediction = Glove_Model(devData.to(device), devDataLength)
            prediction = prediction.cpu()
            prediction = prediction.detach().numpy()
            label = label.detach().numpy()
            devData = devData.detach().numpy()
            prediction = np.argmax(prediction, axis=2)
            prediction = prediction.reshape((len(label), -1))

            for i in range(len(devData)):
                for j in range(len(devData[i])):
                    if devData[i][j] != 0:
                        word = revVocabDict[devData[i][j]]
                        gold = revLabelDict[label[i][j]]
                        op = revLabelDict[prediction[i][j]]
                        devTemp2.write(" ".join([str(j+1), word, gold, op]))
                        devTemp2.write("\n")
                        dev2.write(" ".join([str(j+1), word, op]))
                        dev2.write("\n")
                devTemp2.write("\n")
                dev2.write("\n")
        devTemp2.close()
        dev2.close()

        if epoch == epochs:
          print('model saved')
          torch.save(Glove_Model.state_dict(),'blstm2.pt')

        !perl conll03eval.txt < devtemp2.out

  def __init__(self):
    self.train_lstm()
    self.train_glove()

train()

BiLSTM(
  (embedding): Embedding(26886, 100)
  (LSTM): LSTM(100, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=0.01)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)
Epoch: 1 	Training Loss: 2.286941
{'I-MISC': 0, 'B-PER': 1, 'I-LOC': 2, 'O': 3, 'B-ORG': 4, 'B-LOC': 5, 'I-ORG': 6, 'I-PER': 7, 'B-MISC': 8}
processed 51578 tokens with 5942 phrases; found: 9137 phrases; correct: 1717.
accuracy:  76.16%; precision:  18.79%; recall:  28.90%; FB1:  22.77
              LOC: precision:  35.17%; recall:  43.17%; FB1:  38.76  2255
             MISC: precision:   4.90%; recall:   0.54%; FB1:   0.98  102
              ORG: precision:  90.00%; recall:   0.67%; FB1:   1.33  10
              PER: precision:  13.44%; recall:  49.40%; FB1:  21.13  6770
Epoch: 2 	Training Loss: 1.600799
{'I-MISC': 0, 'B-PER': 1, 'I-LOC': 2, 'O': 3, 'B-ORG': 4, 'B-LOC': 5, 'I-ORG': 

## Testing: 

The class below is used for testing the data. It has 2 functions, t_lstm, which tests the Simple BiLSTM model, and test_glove which tests the Glove model using the testing dataset. \
The outputs are stored in files test1.out and test2.out.

In [ ]:
class test():
  def test_lstm(self):
    BiLSTM_test = BiLSTM_TestLoader(x_test_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    BiLSTM_model = BiLSTM(vocabSize=len(word_idx),
                          embedDimensions=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary))

    BiLSTM_model.load_state_dict(torch.load("./blstm1.pt"))#125
    BiLSTM_model.to(device)

    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    test1 = open("test1.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = BiLSTM_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            test1.write(" ".join([str(j+1), word, op]))
            test1.write("\n")
        test1.write("\n")        
    test1.close()

  def test_glove(self):
    glove = pd.read_csv('./glove.6B.100d.gz', sep=" ",
                        quoting=3, header=None, index_col=0)
    glove_emb = {key: val.values for key, val in glove.T.items()}

    word_idx = getVectors.prepareWordIndex(getVectors(), x_train, x_val)
    glove_vec = np.array([glove_emb[key] for key in glove_emb])
    glove_emb["<PAD>"] = np.zeros((100,), dtype="float64")
    glove_emb["<UNK>"] = np.mean(glove_vec, axis=0, keepdims=True).reshape(100,)
    emb_matrix = getVectors.createEmbedMatrix(getVectors(),
        wordIdx=word_idx, embedDictionary=glove_emb, dimension=100)

    BiLSTM_test = BiLSTM_TestLoader(x_test_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    Glove_model = BiLSTM_Glove(vocabSize=len(word_idx),
                          embedDimension=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary),
                          embedMatrix=emb_matrix)
    
    Glove_model.load_state_dict(torch.load("./blstm2.pt"))#125
    Glove_model.to(device)
    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    test2 = open("test2.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = Glove_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            test2.write(" ".join([str(j+1), word, op]))
            test2.write("\n")
        test2.write("\n")        
    test2.close()

testObj = test()
testObj.test_glove()
testObj.test_lstm() 

In [ ]:
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
import random
import json
from argparse import ArgumentParser
torch.manual_seed(0)
random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

"""Below is a class, createData."""

class createData():
    
  def SplitTrainXY(self, dataset):
      x_train, y_train = list(), list()
      x, y = list(), list()
      first = 1
      for entry in dataset.itertuples():
          # print(type(entry.id))
          # break
          if(entry.ID == '1' and first == 0):
              x_train.append(x)
              y_train.append(y)
              x = list()
              y = list()
          first = 0
          x.append(entry.Word)
          y.append(entry.NERTag)

      x_train.append(x)
      y_train.append(y)

      return x_train, y_train

  def readFileTrain(self, filepath):
      train_df = list()
      with open(filepath, 'r') as inputFile:
          for line in inputFile.readlines():
              if len(line) > 2:
                id, word, ner_tag = line.strip().split(" ")
                train_df.append([id, word, ner_tag])

      train_df = pd.DataFrame(train_df, columns=['ID', 'Word', 'NERTag'])
      train_df = train_df.dropna()
      x_train, y_train = self.SplitTrainXY(train_df)
      return x_train, y_train

  def SplitTestX(self, dataset):
      x_train = list()
      x = list()
      first = 1
      for entry in dataset.itertuples():
          # print(type(entry.id))
          # break
          if(entry.ID == '1' and first == 0):
              x_train.append(x)
              x = list()
          first = 0
          x.append(entry.Word)

      x_train.append(x)
      return x_train

  def readFileTest(self, filepath):
      train_df = list()
      with open(filepath, 'r') as inputFile:
          for line in inputFile.readlines():
              if len(line) > 1:
                  id, word = line.strip().split(" ")
                  train_df.append([id, word])

      train_df = pd.DataFrame(train_df, columns=['ID', 'Word'])
      train_df = train_df.dropna()
      x_train = self.SplitTestX(train_df)
      return x_train


"""Here we have a class getVectors() which is used to generate and return the various vectors and matrices required for training the BiLSTM models."""

class getVectors():

  def prepareVocabulary(self, dataset):
      vocab = set()
      for sentence in dataset:
          for word in sentence:
              vocab.add(word)
      return vocab
  
  def prepareLabelDictionary(self, train_y, val_y):
      label1 = self.prepareVocabulary(train_y)
      label2 = self.prepareVocabulary(val_y)
      label = label1.union(label2)
      label_tuples = []
      counter = 0
      for tags in label:
          label_tuples.append((tags, counter))
          counter += 1
      label_dict = dict(label_tuples)
      return label_dict

  def prepareWordIndex(self, train, val):
      wordIdx = {"<PAD>": 0, "<UNK>": 1}
      idx = 2
      for data in [train, val]:
          for sent in data:
              for word in sent:
                  if word not in wordIdx:
                      wordIdx[word] = idx
                      idx += 1
      return wordIdx

  def vectorizeSentence(self, train, wordIdx):
      train_vec = list()
      tmp_x = list()
      for words in train:
          for word in words:
            if word not in wordIdx.keys():
              tmp_x.append(wordIdx['<UNK>'])
            else:
              tmp_x.append(wordIdx[word])
          train_vec.append(tmp_x)
          tmp_x = list()
      return train_vec

  def vectorizeLabel(self, train_y, label_dict):
      train_y_vec = list()
      for tags in train_y:
          tmp_yy = list()
          for label in tags:
              tmp_yy.append(label_dict[label])
          train_y_vec.append(tmp_yy)
      return train_y_vec
  
  def createEmbedMatrix(self, wordIdx, embedDictionary, dimension):
      embedMatrix = np.zeros((len(wordIdx), dimension))
      for word, idx in wordIdx.items():
          if word in embedDictionary:
              embedMatrix[idx] = embedDictionary[word]
          else:
              if word.lower() in embedDictionary:
                  embedMatrix[idx] = embedDictionary[word.lower()] + 5e-3
              else:
                  embedMatrix[idx] = embedDictionary["<UNK>"]

      return embedMatrix
      
  def initializeClassWeights(self, label_dict, train_y, val_y):
    classWeights = dict()
    for key in label_dict:
        classWeights[key] = 0
    total_nm_tags = 0
    for data in [train_y, val_y]:
        for tags in data:
            for tag in tags:
                total_nm_tags += 1
                classWeights[tag] += 1

    classWt = list()
    for key in classWeights.keys():
        if classWeights[key]:
            score = round(math.log(0.35*total_nm_tags / classWeights[key]), 2)
            classWeights[key] = score if score > 1.0 else 1.0
        else:
            classWeights[key] = 1.0
        classWt.append(classWeights[key])
    classWt = torch.tensor(classWt)
    return classWt



"""## Task 1: Simple Bidirectional LSTM

The class below creates a BiLSTM model with the following parameters and architecture:

Architecture: Embedding layer --> LSTM --> Linear layer --> ELU --> Classifier

Parameters: \
1. Embedding Dimension: 100
2. Number of LSTM layers: 1
3. LSTM Hidden Dimensions: 256
4. LSTM Dropout: 0.33
5. Linear Layer Output Dimensions: 128
"""

class BiLSTM(nn.Module):
    def __init__(self, vocabSize, embedDimensions, linearOutputDimension, hiddenLayerDimension, LSTMLayers,
                 bidirectional, dropoutValue, tagSize):
        super(BiLSTM, self).__init__()

        self.hiddenLayerDimension = hiddenLayerDimension 
        self.LSTMLayers = LSTMLayers 
        self.embedDimensions = embedDimensions 
        self.linearOutputDimension = linearOutputDimension  
        self.tagSize = tagSize 
        if bidirectional:
          self.num_directions = 2 
        else:
          self.num_directions = 1

        """ Initializing Network """
        #Embedding Layer
        self.embedding = nn.Embedding(
            vocabSize, embedDimensions) 
        self.embedding.weight.data.uniform_(-1,1)

        #BiLSTM Layer
        self.LSTM = nn.LSTM(embedDimensions,
                            hiddenLayerDimension,
                            num_layers=LSTMLayers,
                            batch_first=True,
                            bidirectional=True)
        
        #Linear Layer
        self.fc = nn.Linear(hiddenLayerDimension*self.num_directions,
                            linearOutputDimension)  
        
        #Dropout Layer
        self.dropout = nn.Dropout(dropoutValue)

        #ELU Layer
        self.elu = nn.ELU(alpha=0.01)

        #Classifier 
        self.classifier = nn.Linear(linearOutputDimension, self.tagSize)

    def init_hidden(self, batchSize):
        h, c = (torch.zeros(self.LSTMLayers * self.num_directions,
                            batchSize, self.hiddenLayerDimension).to(device),
                torch.zeros(self.LSTMLayers * self.num_directions,
                            batchSize, self.hiddenLayerDimension).to(device))
        return h, c

    def forward(self, sentence, sentenceLength):  
        # Set initial states
        batchSize = sentence.shape[0]
        h_0, c_0 = self.init_hidden(batchSize)

        # Forward propagate LSTM
        embedded = self.embedding(sentence).float()
        packed_embedded = pack_padded_sequence(
            embedded, sentenceLength, batch_first=True, enforce_sorted=False)
        output, _ = self.LSTM(packed_embedded, (h_0, c_0))
        output_unpacked, _ = pad_packed_sequence(output, batch_first=True)
        dropout = self.dropout(output_unpacked)
        linear = self.fc(dropout)
        prediction = self.elu(linear)
        prediction = self.classifier(prediction)
        return prediction

"""## Task 2: Glove word Embeddings

The class below creates a BiLSTM model with the glove vectors with the following parameters and architecture: 

Architecture: Embedding layer --> LSTM --> Linear layer --> ELU --> Classifier

Parameters: \
1. Embedding Dimension: 100
2. Number of LSTM layers: 1
3. LSTM Hidden Dimensions: 256
4. LSTM Dropout: 0.33
5. Linear Layer Output Dimensions: 128
"""

class BiLSTM_Glove(nn.Module):
    def __init__(self, vocabSize, embedDimension, linearOutputDimension, hiddenLayerDimension, LSTMLayers,
                 bidirectional, dropoutValue, tagSize, embedMatrix):
        super(BiLSTM_Glove, self).__init__()
        """ Hyper Parameters """
        self.hiddenLayerDimension = hiddenLayerDimension 
        self.LSTMLayers = LSTMLayers 
        self.embedDimension = embedDimension 
        self.linearOutputDimension = linearOutputDimension 
        self.tagSize = tagSize 
        self.embedMatrix = embedMatrix
        if bidirectional:
          self.numDirections = 2 
        else:
          self.numDirections = 1

        """ Initializing Network """
        #Embedding layer
        self.embedding = nn.Embedding(vocabSize, embedDimension)  
        self.embedding.weight = nn.Parameter(torch.tensor(embedMatrix))

        #LSTM Layer
        self.LSTM = nn.LSTM(embedDimension,
                            hiddenLayerDimension,
                            num_layers=LSTMLayers,
                            batch_first=True,
                            bidirectional=True)
        
        #Linear layer
        self.fc = nn.Linear(hiddenLayerDimension*self.numDirections, linearOutputDimension) 

        #Dropout layer
        self.dropout = nn.Dropout(dropoutValue)

        #ELU Layer
        self.elu = nn.ELU(alpha=0.01)

        #CLassifier
        self.classifier = nn.Linear(linearOutputDimension, self.tagSize)

    def init_hidden(self, batch_size):
        h, c = (torch.zeros(self.LSTMLayers * self.numDirections,
                            batch_size, self.hiddenLayerDimension).to(device),
                torch.zeros(self.LSTMLayers * self.numDirections,
                            batch_size, self.hiddenLayerDimension).to(device))
        return h, c

    def forward(self, sentence, sentenceLength):  
        #Set initial states
        batch_size = sentence.shape[0]
        h_0, c_0 = self.init_hidden(batch_size)

        # Forward propagate LSTM
        embedded = self.embedding(sentence).float()
        packed_embedded = pack_padded_sequence(embedded, sentenceLength, batch_first=True, enforce_sorted=False)
        output, _ = self.LSTM(packed_embedded, (h_0, c_0))
        output_unpacked, _ = pad_packed_sequence(output, batch_first=True)
        dropout = self.dropout(output_unpacked)
        linear = self.fc(dropout)
        prediction = self.elu(linear)
        prediction = self.classifier(prediction)
        return prediction

"""The two classes below, BiLSTM_DataLoader and BiLSTM_TestLoader load the dataset by converting the data into torch tensors. """

class BiLSTM_DataLoader(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        xInstance = torch.tensor(self.x[index])  # , dtype=torch.long
        yInstance = torch.tensor(self.y[index])  # , dtype=torch.float
        return xInstance, yInstance

class BiLSTM_TestLoader(Dataset):
    def __init__(self, x):
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        xInstance = torch.tensor(self.x[index])  # , dtype=torch.long
        return xInstance

"""The two classes below, CustomCollator and CustomTestCollator both implement padding for the input data by considering the max padding length as the max length in the Batch. """

class CustomCollator(object):

    def __init__(self, vocabulary, label):
        self.params = vocabulary
        self.label = label

    def __call__(self, batch):
        (xx, yy) = zip(*batch)
        xLength = [len(x) for x in xx]
        ylength = [len(y) for y in yy]
        maxBatchlength = max([len(s) for s in xx])
        batchData = self.params['<PAD>']*np.ones((len(xx), maxBatchlength))
        batchLabels = -1*np.zeros((len(xx), maxBatchlength))
        for j in range(len(xx)):
            cur_len = len(xx[j])
            batchData[j][:cur_len] = xx[j]
            batchLabels[j][:cur_len] = yy[j]

        batchData, batchLabels = torch.LongTensor(
            batchData), torch.LongTensor(batchLabels)
        batchData, batchLabels = Variable(batchData), Variable(batchLabels)

        return batchData, batchLabels, xLength, ylength

class CustomTestCollator(object):

    def __init__(self, vocabulary, label):
        self.params = vocabulary
        self.label = label

    def __call__(self, batch):
        xx = batch
        xLen = [len(x) for x in xx]
        maxBatchlength = max([len(s) for s in xx])
        batchData = self.params['<PAD>']*np.ones((len(xx), maxBatchlength))
        for j in range(len(xx)):
            cur_len = len(xx[j])
            batchData[j][:cur_len] = xx[j]

        batchData = torch.LongTensor(batchData)
        batchData = Variable(batchData)

        return batchData, xLen

"""## Training:

The class below is used for training the data. It has 2 functions, train_lstm, which trains the Simple BiLSTM model, and train_glove which trains the Glove model using the training and dev datasets. 

1. Simple BiLSTM Training: \
It is trained using the following parameters: \
  *   batch size: 4
  *   optimizer: SGD, with learning_rate = 0.1 and momentum = 0.9
  *   scheduler: StepLR, with step_size = 20 and gamma = 0.9

2. BiLSTM Training using Glove Embeddings: \
It is trained using the following parameters: \
  *   batch size: 8
  *   optimizer: SGD, with learning_rate = 0.1 and momentum = 0.9
  *   scheduler: StepLR, with step_size = 15 and gamma = 0.9

The dev outputs are stored in 2 files, dev1.out and dev2.out.

"""

class train():

  def train_lstm(self):
    BiLSTM_model = BiLSTM(vocabSize=len(word_idx),
                          embedDimensions=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary))
    BiLSTM_model.to(device)
    print(BiLSTM_model)

    BiLSTM_train = BiLSTM_DataLoader(x_train_vectors, y_train_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader = DataLoader(dataset=BiLSTM_train,
                            batch_size=4,
                            drop_last=True,
                            collate_fn=custom_collator)

    BiLSTM_dev = BiLSTM_DataLoader(x_val_vectors, y_val_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader_dev = DataLoader(dataset=BiLSTM_dev,
                                batch_size=1,
                                shuffle=False,
                                drop_last=True,
                                collate_fn=custom_collator)

    criteria = nn.CrossEntropyLoss(weight=class_wt)
    criteria = criteria.to(device)
    criteria.requres_grad = True
    optimizer = torch.optim.SGD(BiLSTM_model.parameters(), lr=0.1, momentum=0.9)
    scheduler = StepLR(optimizer, step_size = 20, gamma=0.9)
    epochs = 50

    for epoch in range(1, epochs+1):
        train_loss = 0.0
        for input, label, inputLength, labelLength in dataloader:
            optimizer.zero_grad()
            output = BiLSTM_model(input.to(device), inputLength)
            output = output.view(-1, len(label_dictionary))
            label = label.view(-1)
            loss = criteria(output, label.to(device))
            loss.backward()
            train_loss += loss.item() * input.size(1)
            optimizer.step()

        train_loss = train_loss / len(dataloader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        #torch.save(BiLSTM_model.state_dict(),'BiLSTM_b1_epoch_' + str(i) + '.pt')
        #BiLSTM_model.load_state_dict(torch.load("./BiLSTM_b1_epoch_"+str(e)+".pt"))#125
        BiLSTM_model.to(device)
        print(label_dictionary)
        revLabelDict = {v: k for k, v in label_dictionary.items()}
        revVocabDict = {v: k for k, v in word_idx.items()}

        #devTemp = open("./devtemp.out", 'w')
        dev1 = open("./dev1.out", 'w')
        val_loss = 0.0
        for devData, label, devDataLength, labelDataLength in dataloader_dev:

            prediction = BiLSTM_model(devData.to(device), devDataLength)

            output = prediction.view(-1, len(label_dictionary))
            label2 = label.view(-1)
            loss = criteria(output, label2.to(device))
            loss.backward()
            val_loss += loss.item() * devData.size(1)

            prediction = prediction.cpu()
            prediction = prediction.detach().numpy()
            label = label.detach().numpy()
            devData = devData.detach().numpy()
            prediction = np.argmax(prediction, axis=2)
            prediction = prediction.reshape((len(label), -1))

            for i in range(len(devData)):
                for j in range(len(devData[i])):
                    if devData[i][j] != 0:
                        word = revVocabDict[devData[i][j]]
                        gold = revLabelDict[label[i][j]]
                        op = revLabelDict[prediction[i][j]]
                        #devTemp.write(" ".join([str(j+1), word, gold, op]))
                        #devTemp.write("\n")
                        dev1.write(" ".join([str(j+1), word, op]))
                        dev1.write("\n")
                #devTemp.write("\n")
                dev1.write("\n")
        #devTemp.close()
        dev1.close()

        scheduler.step()

        if epoch == epochs:
          #print('model saved')
          torch.save(BiLSTM_model.state_dict(),'blstm1.pt')

        #!perl conll03eval.txt < devtemp.out

  def train_glove(self):

    glove = pd.read_csv('./glove.6B.100d.gz', sep=" ",
                        quoting=3, header=None, index_col=0)
    glove_emb = {key: val.values for key, val in glove.T.items()}

    word_idx = getVectors.prepareWordIndex(getVectors(), x_train, x_val)
    glove_vec = np.array([glove_emb[key] for key in glove_emb])
    glove_emb["<PAD>"] = np.zeros((100,), dtype="float64")
    glove_emb["<UNK>"] = np.mean(glove_vec, axis=0, keepdims=True).reshape(100,)
    emb_matrix = getVectors.createEmbedMatrix(getVectors(),
        wordIdx=word_idx, embedDictionary=glove_emb, dimension=100)

    vocab_size = emb_matrix.shape[0]
    vector_size = emb_matrix.shape[1]
    print(vocab_size, vector_size)
    Glove_Model = BiLSTM_Glove(vocabSize=len(word_idx),
                          embedDimension=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary),
                          embedMatrix=emb_matrix)
    Glove_Model.to(device)
    print(Glove_Model)

    BiLSTM_train = BiLSTM_DataLoader(x_train_vectors, y_train_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader = DataLoader(dataset=BiLSTM_train,
                            batch_size=8,
                            drop_last=True,
                            collate_fn=custom_collator)

    BiLSTM_dev = BiLSTM_DataLoader(x_val_vectors, y_val_vectors)
    custom_collator = CustomCollator(word_idx, label_dictionary)
    dataloader_dev = DataLoader(dataset=BiLSTM_dev,
                                batch_size=1,
                                shuffle=False,
                                drop_last=True,
                                collate_fn=custom_collator)

    criterion = nn.CrossEntropyLoss(weight=class_wt)
    criterion = criterion.to(device)
    criterion.requres_grad = True
    optimizer = torch.optim.SGD(Glove_Model.parameters(), lr=0.1, momentum=0.9)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.9)
    epochs = 25

    for epoch in range(1, epochs+1):
        train_loss = 0.0
        for input, label, input_len, label_len in dataloader:
            optimizer.zero_grad()
            output = Glove_Model(input.to(device), input_len)
            output = output.view(-1, len(label_dictionary))
            label = label.view(-1)
            loss = criterion(output, label.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * input.size(1)

        train_loss = train_loss / len(dataloader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

        #torch.save(Glove_Model.state_dict(),'BiLSTM_glove_' + str(i) + '.pt')
        #Glove_Model.load_state_dict(torch.load("./BiLSTM_glove_"+str(e)+".pt"))#125

        Glove_Model.to(device)
        revLabelDict = {v: k for k, v in label_dictionary.items()}
        revVocabDict = {v: k for k, v in word_idx.items()}
        
        #devTemp2 = open("devtemp2.out", 'w')
        dev2 = open("dev2.out", 'w')

        for devData, label, devDataLength, label_data_len in dataloader_dev:

            prediction = Glove_Model(devData.to(device), devDataLength)
            prediction = prediction.cpu()
            prediction = prediction.detach().numpy()
            label = label.detach().numpy()
            devData = devData.detach().numpy()
            prediction = np.argmax(prediction, axis=2)
            prediction = prediction.reshape((len(label), -1))

            for i in range(len(devData)):
                for j in range(len(devData[i])):
                    if devData[i][j] != 0:
                        word = revVocabDict[devData[i][j]]
                        gold = revLabelDict[label[i][j]]
                        op = revLabelDict[prediction[i][j]]
                        #devTemp2.write(" ".join([str(j+1), word, gold, op]))
                        #devTemp2.write("\n")
                        dev2.write(" ".join([str(j+1), word, op]))
                        dev2.write("\n")
                #devTemp2.write("\n")
                #dev2.write("\n")
        #devTemp2.close()
        dev2.close()

        if epoch == epochs:
          #print('model saved')
          torch.save(Glove_Model.state_dict(),'blstm2.pt')

        #!perl conll03eval.txt < devtemp2.out

"""## Testing: 

The class below is used for testing the data. It has 2 functions, t_lstm, which tests the Simple BiLSTM model, and test_glove which tests the Glove model using the testing dataset.
The outputs are stored in files test1.out and test2.out.
"""

class test():
  def test_lstm(self):
    BiLSTM_test = BiLSTM_TestLoader(x_test_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    BiLSTM_model = BiLSTM(vocabSize=len(word_idx),
                          embedDimensions=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary))

    BiLSTM_model.load_state_dict(torch.load("./blstm1.pt"))#125
    BiLSTM_model.to(device)

    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    test1 = open("test1.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = BiLSTM_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            test1.write(" ".join([str(j+1), word, op]))
            test1.write("\n")
        test1.write("\n")        
    test1.close()

  def test_glove(self):
    glove = pd.read_csv('./glove.6B.100d.gz', sep=" ",
                        quoting=3, header=None, index_col=0)
    glove_emb = {key: val.values for key, val in glove.T.items()}

    word_idx = getVectors.prepareWordIndex(getVectors(), x_train, x_val)
    glove_vec = np.array([glove_emb[key] for key in glove_emb])
    glove_emb["<PAD>"] = np.zeros((100,), dtype="float64")
    glove_emb["<UNK>"] = np.mean(glove_vec, axis=0, keepdims=True).reshape(100,)
    emb_matrix = getVectors.createEmbedMatrix(getVectors(),
        wordIdx=word_idx, embedDictionary=glove_emb, dimension=100)

    BiLSTM_test = BiLSTM_TestLoader(x_test_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    Glove_model = BiLSTM_Glove(vocabSize=len(word_idx),
                          embedDimension=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary),
                          embedMatrix=emb_matrix)
    
    Glove_model.load_state_dict(torch.load("./blstm2.pt"))#125
    Glove_model.to(device)
    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    test2 = open("test2.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = Glove_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            test2.write(" ".join([str(j+1), word, op]))
            test2.write("\n")
        test2.write("\n")        
    test2.close()

  def dev_lstm(self):
    BiLSTM_test = BiLSTM_TestLoader(x_val_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    BiLSTM_model = BiLSTM(vocabSize=len(word_idx),
                          embedDimensions=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary))

    BiLSTM_model.load_state_dict(torch.load("./blstm1.pt"))#125
    BiLSTM_model.to(device)

    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    dev1 = open("dev1.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = BiLSTM_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            dev1.write(" ".join([str(j+1), word, op]))
            dev1.write("\n")
        dev1.write("\n")        
    dev1.close()

  def dev_glove(self):
    glove = pd.read_csv('./glove.6B.100d.gz', sep=" ",
                        quoting=3, header=None, index_col=0)
    glove_emb = {key: val.values for key, val in glove.T.items()}

    word_idx = getVectors.prepareWordIndex(getVectors(), x_train, x_val)
    glove_vec = np.array([glove_emb[key] for key in glove_emb])
    glove_emb["<PAD>"] = np.zeros((100,), dtype="float64")
    glove_emb["<UNK>"] = np.mean(glove_vec, axis=0, keepdims=True).reshape(100,)
    emb_matrix = getVectors.createEmbedMatrix(getVectors(),
        wordIdx=word_idx, embedDictionary=glove_emb, dimension=100)

    BiLSTM_test = BiLSTM_TestLoader(x_val_vectors)
    custom_test_collator = CustomTestCollator(word_idx, label_dictionary)
    dataloader_test = DataLoader(dataset=BiLSTM_test,
                                    batch_size=1,
                                    shuffle=False,
                                    drop_last=True,
                                    collate_fn=custom_test_collator)
    
    Glove_model = BiLSTM_Glove(vocabSize=len(word_idx),
                          embedDimension=100,
                          linearOutputDimension=128,
                          hiddenLayerDimension=256,
                          LSTMLayers=1,
                          bidirectional=True,
                          dropoutValue=0.33,
                          tagSize=len(label_dictionary),
                          embedMatrix=emb_matrix)
    
    Glove_model.load_state_dict(torch.load("./blstm2.pt"))#125
    Glove_model.to(device)
    revLabelDict = {v: k for k, v in label_dictionary.items()}
    revVocabDict = {v: k for k, v in word_idx.items()}

    dev2 = open("dev2.out", 'w')
    for testData, testDataLength in dataloader_test:
      prediction = Glove_model(testData.to(device), testDataLength)
      prediction = prediction.cpu()
      prediction = prediction.detach().numpy()
      testData = testData.detach().numpy()
      prediction = np.argmax(prediction, axis=2)
      prediction = prediction.reshape((len(testData), -1))
            
      for i in range(len(testData)):
        for j in range(len(testData[i])):
          if testData[i][j] != 0:
            word = revVocabDict[testData[i][j]]
            op = revLabelDict[prediction[i][j]]
            dev2.write(" ".join([str(j+1), word, op]))
            dev2.write("\n")
        dev2.write("\n")        
    dev2.close()

if __name__ == '__main__':
    createObj = createData()
    x_train, y_train = createObj.readFileTrain('./data/train')
    x_val, y_val = createObj.readFileTrain('./data/dev')
    x_test = createObj.readFileTest('./data/test')

    vectorObj = getVectors()
    word_idx = vectorObj.prepareWordIndex(x_train, x_val)
    x_train_vectors = vectorObj.vectorizeSentence(x_train, word_idx)
    x_test_vectors = vectorObj.vectorizeSentence(x_test, word_idx)
    x_val_vectors = vectorObj.vectorizeSentence(x_val, word_idx)
    label_dictionary = vectorObj.prepareLabelDictionary(y_train, y_val)
    y_train_vectors = vectorObj.vectorizeLabel(y_train, label_dictionary)
    y_val_vectors = vectorObj.vectorizeLabel(y_val, label_dictionary)

    class_wt = vectorObj.initializeClassWeights(label_dictionary, y_train, y_val)
    
    
    #parser = ArgumentParser()
    #parser.add_argument("--mode", "-m", type=str, choices=['train', 'test'], default='test')
    args = 'test'
    
    if args == 'train':
        trainObj = train()
        trainObj.train_lstm()
        trainObj.train_glove()
    
    elif args == 'test':
        testObj = test()
        testObj.test_glove()
        testObj.test_lstm()
        testObj.dev_lstm()
        testObj.dev_glove()
